In [18]:
import numpy as np
from PIL import Image
import re
from src.css.css_parser import *
from fpdf import FPDF
from src.css.color_utils import *
import os.path
from src.fpdf_extention import *
from src.md_to_pdf_converter import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
colors = get_theme_from_file("Obsidian Nord.css")

In [95]:
pool = StylePool()
pool.doc_style.set_background_color(colors["--background-primary"])
pool.style_normal.set_font_color(colors["--text-normal"])

pool.style_h1.set_font_color(colors["--text-title-h1"])
pool.style_h2.set_font_color(colors["--text-title-h2"])
pool.style_h3.set_font_color(colors["--text-title-h3"])
pool.style_h4.set_font_color(colors["--text-title-h4"])
pool.style_h5.set_font_color(colors["--text-title-h5"])
pool.style_h6.set_font_color(colors["--text-title-h6"])

pool.style_code.set_font_color(colors["--code-block"])
pool.style_code.set_background_color(colors["--pre-code"])

pool.style_mark.set_font_color(colors["--text-normal"])
pool.style_mark.set_background_color(colors["--pre-code"])

pool.style_link.set_font_color(colors["--text-accent"])


In [125]:
file_text = read_contents("MDtest.md")

In [109]:
pdf = PrintablePDF()
convert_md_to_pdf(file_text, pdf, pool)
#pdf.link(10,10,100,100, 'http|://www.intranet.com/mb/rprh06/final.php?folio=')

pdf.output("mygfg.pdf")

''

In [110]:
text = "[link to the page](https://github.com/Altair200333/ObsidianMDtoPDF)"

regex = re.compile(r"\[(.*)\]\((.*)\)")
matches = regex.findall(text)

matches

[('link to the page', 'https://github.com/Altair200333/ObsidianMDtoPDF')]

In [130]:
#pdf = PrintablePDF()

blocks = [H1Block(pool), H2Block(pool), H3Block(pool), H4Block(pool), H5Block(pool), H6Block(pool), LinkBlock(pool)]

def mark_block(block, id, table, string):
    results = [(m.start(0), m.end(0)) for m in block.regex.finditer(string)]
    for res in results:
        table[res[0]] = id
        table[res[1]] = -id
        print(string[res[0]:res[1]])
        
style_dict = {}

for i, block in enumerate(blocks):
    mark_block(block, i + 1, style_dict, file_text)

print(style_dict)

style_stack = []
for i, c in enumerate(file_text):
    if i in style_dict:
        value = style_dict[i]
        if value > 0:
            print("start style", abs(value))
            style_stack.append(value)
        if value < 0:
            print("end style", abs(value))
            style_stack.pop()
        print(style_stack)

#pdf.output("mygfg.pdf")

# Title 1
[link to the page](https://github.com/Altair200333/ObsidianMDtoPDF)
[link to the page](https://github.com/Altair200333/ObsidianMDtoPDF5)
[link to the page](https://github.com/Altair200333/ObsidianMDtoPDF3)
[link to the page](https://github.com/Altair200333/ObsidianMDtoPDF2)
{0: 1, 9: -1, 76: 7, 143: -7, 144: 7, 212: -7, 214: 7, 282: -7, 284: 7, 352: -7}
start style 1
[1]
end style 1
[]
start style 7
[7]
end style 7
[]
start style 7
[7]
end style 7
[]
start style 7
[7]
end style 7
[]
start style 7
[7]
end style 7
[]


In [134]:
results = [(m.start(0), m.end(0)) for m in H2Block(pool).regex.finditer(file_text)]
results

[]

In [154]:
res = re.findall(r"## .*", "## Title 2")
res

[]